In [2]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [7]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\ESTADIC")

In [35]:
df18=pd.read_csv("ESTADIC_educacao_2018.csv", sep=';')
df21=pd.read_csv("ESTADIC_educacao_2021.csv", sep=';')

In [36]:
df18 = df18[['Cod UF', 'uf', 'genero', 'idade', 'cor_raca',
        'escolaridade']]
df21 = df21[['Cod UF', 'uf', 'genero', 'idade', 'cor_raca',
        'escolaridade']]
        

In [37]:
df18['ano']=2018
df21['ano']=2021

In [69]:
df = pd.concat([df18,df21])

In [70]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [57]:
x = df.pivot_table(columns=['ano','genero', 'faixa_etaria', 'cor_raca',
        'escolaridade'], aggfunc='size')
y = pd.DataFrame(x)
y = y.reset_index() ##inserir a contagem como coluna
y = y.rename(columns={0:'count'})

In [59]:
df = y[y['count']>0]

In [ ]:
### Subindo para o GBQ

In [60]:
client = bigquery.Client()

In [61]:
dataset_ref = client.dataset('cargos_lideranca')

In [63]:
df=df.rename(columns={'count':'quantidade'})

In [64]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
 bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
 bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('escolaridade','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
 bigquery.SchemaField('quantidade','INTEGER',description='Área deformação.')
 ]

In [65]:
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_educacao_consolidado')

In [66]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [68]:
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=28b5cbbd-3cd9-455e-8d2c-62130abcdd5a>